## 라이브러리

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import autokeras as ak
from sklearn.model_selection import train_test_split
# import jinja2
# from pycaret.classification import *

In [7]:
df = pd.read_csv('../dataset/master_3.csv')
df.head()

,index,subject_id,stay_id,intime,outtime,gender,in_year,age,outcome_inhospital_mortality,ed_death,before_ed_mortality,ed_los,outcome_icu_transfer_12h,outcome_hospitalization,outcome_critical,n_ed_30d,n_ed_90d,n_ed_365d,outcome_ed_revisit_3d,n_hosp_30d,n_hosp_90d,n_hosp_365d,n_icu_30d,n_icu_90d,n_icu_365d,ed_los_hours,triage_temperature,triage_heartrate,triage_resprate,triage_o2sat,triage_sbp,triage_dbp,triage_pain,triage_acuity,chiefcom_chest_pain,chiefcom_abdominal_pain,chiefcom_headache,chiefcom_shortness_of_breath,chiefcom_back_pain,chiefcom_cough,chiefcom_nausea_vomiting,chiefcom_fever_chills,chiefcom_syncope,chiefcom_dizziness,cci_MI,cci_CHF,cci_PVD,cci_Stroke,cci_Dementia,cci_Pulmonary,cci_Rheumatic,cci_PUD,cci_Liver1,cci_DM1,cci_DM2,cci_Paralysis,cci_Renal,cci_Cancer1,cci_Liver2,cci_Cancer2,cci_HIV,eci_CHF,eci_Arrhythmia,eci_Valvular,eci_PHTN,eci_PVD,eci_HTN1,eci_HTN2,eci_Paralysis,eci_NeuroOther,eci_Pulmonary,eci_DM1,eci_DM2,eci_Hypothyroid,eci_Renal,eci_Liver,eci_PUD,eci_HIV,eci_Lymphoma,eci_Tumor2,eci_Tumor1,eci_Rheumatic,eci_Coagulopathy,eci_Obesity,eci_WeightLoss,eci_FluidsLytes,eci_BloodLoss,eci_Anemia,eci_Alcohol,eci_Drugs,eci_Psychoses,eci_Depression,ed_temperature_last,ed_heartrate_last,ed_resprate_last,ed_o2sat_last,ed_sbp_last,ed_dbp_last,ed_pain_last,n_med,n_medrecon
0,335314,10000032,33258284,2180-05-06 19:17:00,2180-05-06 23:30:00,0,2180,52,False,False,False,0 days 04:13:00,False,True,False,0,0,0,False,0,0,0,0,0,0,4.22,36.888889,70.0,16.0,97.0,106.0,63.0,0.0,3.0,False,True,False,False,False,False,False,False,False,False,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,36.500000,79.0,16.0,98.0,107.0,60.0,0.0,0.0,9.0
1,335315,10000032,38112554,2180-06-26 15:54:00,2180-06-26 21:31:00,0,2180,52,False,False,False,0 days 05:37:00,False,True,False,0,1,1,False,0,1,1,0,0,0,5.62,37.166667,88.0,18.0,97.0,116.0,88.0,10.0,3.0,False,False,False,False,False,False,False,False,False,False,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,36.611111,86.0,17.0,93.0,96.0,57.0,5.0,3.0,12.0
2,335316,10000032,32952584,2180-07-22 16:24:00,2180-07-23 05:54:00,0,2180,52,False,False,False,0 days 13:30:00,True,True,True,1,2,2,True,1,2,2,0,0,0,13.50,36.555556,87.0,14.0,97.0,71.0,43.0,7.0,2.0,False,False,False,False,False,False,False,False,False,False,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,36.777778,85.0,18.0,98.0,81.0,38.0,0.0,2.0,14.0
3,335317,10000032,39399961,2180-07-23 05:54:00,2180-07-23 14:00:00,0,2180,52,False,False,False,0 days 08:06:00,True,True,True,2,3,3,False,1,2,2,0,0,0,8.10,37.055556,77.0,16.0,98.0,96.0,50.0,4.0,2.0,False,True,False,False,False,False,False,False,False,False,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,37.222222,96.0,18.0,97.0,86.0,45.0,0.0,8.0,14.0
4,335318,10000032,35968195,2180-08-05 20:58:00,2180-08-06 01:44:00,0,2180,52,False,False,False,0 days 04:46:00,False,True,False,2,3,4,False,1,2,3,1,1,1,4.77,37.444444,105.0,18.0,96.0,106.0,57.0,10.0,3.0,False,True,False,False,False,False,False,False,False,False,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,1,36.722222,91.0,18.0,99.0,98.0,60.0,0.0,6.0,7.0


In [ ]:
df.columns

Index(['index', 'subject_id', 'stay_id', 'intime', 'outtime', 'gender',
       'in_year', 'age', 'outcome_inhospital_mortality', 'ed_death',
       ...
       'eci_Depression', 'ed_temperature_last', 'ed_heartrate_last',
       'ed_resprate_last', 'ed_o2sat_last', 'ed_sbp_last', 'ed_dbp_last',
       'ed_pain_last', 'n_med', 'n_medrecon'],
      dtype='object', length=101)

In [8]:
drop_df = df.drop(list(df.filter(regex='cci_').columns), axis=1)

In [9]:
drop_df = drop_df.drop(['outcome_icu_transfer_12h'], axis=1)

In [10]:
drop_df = drop_df.drop(['chiefcom_chest_pain', 'chiefcom_abdominal_pain',
    'chiefcom_headache', 'chiefcom_shortness_of_breath',
    'chiefcom_back_pain', 'chiefcom_cough', 'chiefcom_nausea_vomiting',
    'chiefcom_fever_chills', 'chiefcom_syncope', 'chiefcom_dizziness',
    'eci_CHF', 'eci_Arrhythmia', 'eci_Valvular', 'eci_PHTN', 'eci_PVD',
    'eci_HTN1', 'eci_HTN2', 'eci_Paralysis', 'eci_NeuroOther',
    'eci_Pulmonary', 'eci_DM1', 'eci_DM2', 'eci_Hypothyroid', 'eci_Renal',
    'eci_Liver', 'eci_PUD', 'eci_HIV', 'eci_Lymphoma', 'eci_Tumor2',
    'eci_Tumor1', 'eci_Rheumatic', 'eci_Coagulopathy', 'eci_Obesity',
    'eci_WeightLoss', 'eci_FluidsLytes', 'eci_BloodLoss', 'eci_Anemia',
    'eci_Alcohol', 'eci_Drugs', 'eci_Psychoses', 'eci_Depression'], axis=1)

## 각각 모델링

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.datasets import make_classification
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from collections import Counter

In [24]:
obj = pd.DataFrame(drop_df.dtypes != 'object')
drop_df = drop_df[obj[obj[0] == True].index]

In [67]:
drop_df['outcome_critical'].value_counts()

0    419387
1     29585
Name: outcome_critical, dtype: int64

In [25]:
y = drop_df['outcome_critical']
x = drop_df.drop(['outcome_critical'], axis=1)

In [26]:
#언더 샘플링
undersample = RandomUnderSampler(sampling_strategy='majority')
x_under, y_under = undersample.fit_resample(x, y)
print(Counter(y_under))

Counter({False: 29585, True: 29585})


In [38]:
x_train, x_test, y_train, y_test = train_test_split(x_under, y_under, shuffle=True, stratify=y_under, test_size=0.2, random_state=10)

In [100]:
# 오버샘플링
sm = SMOTE(random_state=10)

# train데이터를 넣어 복제함
x_smote, y_smote = sm.fit_sample(data, target)

print('After OverSampling, the shape of train_X: {}'.format(x_smote.shape))
print('After OverSampling, the shape of train_y: {} \n'.format(x_smote.shape))

# print("After OverSampling, counts of label '1': {}".format(sum(y_smote==True)))
# print("After OverSampling, counts of label '0': {}".format(sum(y_smote==False)))

After OverSampling, the shape of train_X: (838774, 38)
After OverSampling, the shape of train_y: (838774, 38) 



In [95]:
x_train, x_test, y_train, y_test = train_test_split(x_smote, y_smote, shuffle=True, stratify=y_smote, test_size=0.2, random_state=10)

In [39]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, 
                                                  test_size=0.1, stratify=y_train, 
                                                  random_state=10)

In [39]:
# scaler = MinMaxScaler()
# x_train2 = scaler.fit_transform(x_train)
# x_test2 = scaler.transform(x_test)
# x_val2 = scaler.transform(x_val)

In [2]:
train = pd.read_csv('../dataset/train_D_U_O.csv').iloc[:,1:]
test = pd.read_csv('../dataset/test_D_U_O.csv').iloc[:,1:]
X = train.drop(['outcome_critical'], axis=1)
y = train['outcome_critical']
X_test = test.drop(['outcome_critical'], axis=1)
y_test = test['outcome_critical']
# Split dataset into training set and test set
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=2022) # 80% training and 20% test

In [5]:
lr_list = [0.05, 0.075, 0.1, 0.25, 0.5, 0.75, 1]

for learning_rate in lr_list:
    gb_clf = GradientBoostingClassifier(n_estimators=20, learning_rate=learning_rate, max_features=2, max_depth=2, random_state=0)
    gb_clf.fit(X_train, y_train)

    print("Learning rate: ", learning_rate)
    print("Accuracy score (training): {0:.3f}".format(gb_clf.score(X_train, y_train)))
    print("Accuracy score (validation): {0:.3f}".format(gb_clf.score(X_val, y_val)))

Learning rate:  0.05
Accuracy score (training): 0.823
Accuracy score (validation): 0.815
Learning rate:  0.075
Accuracy score (training): 0.834
Accuracy score (validation): 0.828
Learning rate:  0.1
Accuracy score (training): 0.840
Accuracy score (validation): 0.836
Learning rate:  0.25
Accuracy score (training): 0.867
Accuracy score (validation): 0.861
Learning rate:  0.5
Accuracy score (training): 0.878
Accuracy score (validation): 0.872
Learning rate:  0.75
Accuracy score (training): 0.880
Accuracy score (validation): 0.875
Learning rate:  1
Accuracy score (training): 0.880
Accuracy score (validation): 0.874


In [11]:
for features in range(2, 10, 2):
    gb_clf = GradientBoostingClassifier(n_estimators=20, learning_rate=0.75, max_features=features, max_depth=2, random_state=2022)
    gb_clf.fit(X_train, y_train)

    print("features: ", features)
    print("Accuracy score (training): {0:.3f}".format(gb_clf.score(X_train, y_train)))
    print("Accuracy score (validation): {0:.3f}".format(gb_clf.score(X_val, y_val)))

features:  2
Accuracy score (training): 0.887
Accuracy score (validation): 0.882
features:  4
Accuracy score (training): 0.889
Accuracy score (validation): 0.884
features:  6
Accuracy score (training): 0.887
Accuracy score (validation): 0.881
features:  8
Accuracy score (training): 0.892
Accuracy score (validation): 0.887


In [10]:
lr_list = [0.05, 0.075, 0.1, 0.25, 0.5, 0.75, 1]

for depth in range(4, 16, 2):
    gb_clf = GradientBoostingClassifier(n_estimators=20, learning_rate=0.75, max_features=2, max_depth=depth, random_state=2022)
    gb_clf.fit(X_train, y_train)

    print("depth: ", depth)
    print("Accuracy score (training): {0:.3f}".format(gb_clf.score(X_train, y_train)))
    print("Accuracy score (validation): {0:.3f}".format(gb_clf.score(X_val, y_val)))

depth:  4
Accuracy score (training): 0.895
Accuracy score (validation): 0.890
depth:  6
Accuracy score (training): 0.908
Accuracy score (validation): 0.894
depth:  8
Accuracy score (training): 0.924
Accuracy score (validation): 0.892
depth:  10
Accuracy score (training): 0.947
Accuracy score (validation): 0.895
depth:  12
Accuracy score (training): 0.970
Accuracy score (validation): 0.890
depth:  14
Accuracy score (training): 0.997
Accuracy score (validation): 0.887


In [18]:
for est in range(10, 32, 4):
    gb_clf = GradientBoostingClassifier(n_estimators=est, learning_rate=0.75, max_features=features, max_depth=2, random_state=2022)
    gb_clf.fit(X_train, y_train)

    print("estimators: ", est)
    print("Accuracy score (training): {0:.3f}".format(gb_clf.score(X_train, y_train)))
    print("Accuracy score (validation): {0:.3f}".format(gb_clf.score(X_val, y_val)))

estimators:  10
Accuracy score (training): 0.877
Accuracy score (validation): 0.872
estimators:  14
Accuracy score (training): 0.884
Accuracy score (validation): 0.879
estimators:  18
Accuracy score (training): 0.889
Accuracy score (validation): 0.883
estimators:  22
Accuracy score (training): 0.893
Accuracy score (validation): 0.887
estimators:  26
Accuracy score (training): 0.896
Accuracy score (validation): 0.889
estimators:  30
Accuracy score (training): 0.898
Accuracy score (validation): 0.892


In [6]:
gb_clf2 = GradientBoostingClassifier(n_estimators=20, learning_rate=0.75, max_features=2, max_depth=12, random_state=2022)
gb_clf2.fit(X_train, y_train)
predictions = gb_clf2.predict(X_val)

print("Confusion Matrix:")
print(confusion_matrix(y_val, predictions))

print("Classification Report")
print(classification_report(y_val, predictions))
print("Accuracy score (training): {0:.3f}".format(gb_clf2.score(X_train, y_train)))
print("Accuracy score (test): {0:.3f}".format(gb_clf2.score(X_test, y_test)))

Confusion Matrix:
[[3694  529]
 [ 395 3809]]
Classification Report
              precision    recall  f1-score   support

       False       0.90      0.87      0.89      4223
        True       0.88      0.91      0.89      4204

    accuracy                           0.89      8427
   macro avg       0.89      0.89      0.89      8427
weighted avg       0.89      0.89      0.89      8427

Accuracy score (training): 0.970
Accuracy score (test): 0.874


In [7]:
from sklearn.metrics import accuracy_score, recall_score, f1_score, precision_score, roc_auc_score, confusion_matrix

def get_clf_eval(y_test, y_pred):
    confusion = confusion_matrix(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    F1 = f1_score(y_test, y_pred)
    AUC = roc_auc_score(y_test, y_pred)
    print('오차행렬:\n', confusion)
    print('\n정확도: {:.4f}'.format(accuracy))
    print('정밀도: {:.4f}'.format(precision))
    print('재현율: {:.4f}'.format(recall))
    print('F1: {:.4f}'.format(F1))
    print('AUC: {:.4f}'.format(AUC))    

In [9]:
pred = gb_clf2.predict(X_test)
get_clf_eval(y_test, pred)

오차행렬:
 [[73631 10268]
 [ 1022  4874]]

정확도: 0.8743
정밀도: 0.3219
재현율: 0.8267
F1: 0.4634
AUC: 0.8521
